### Import Library

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

### Define Helper Functions

In [ ]:
def generate_ifn(value):
    # Generate a random membership degree between 0 and value/4
    # m = np.random.uniform(0, value/4)

    # # Ensure the non-membership degree satisfies m + n <= 1
    # n = np.random.uniform(0, 1 - m)
    ifns = {
        "0":(0.10,0.90),
        "1":(0.20,0.65),
        "2":(0.30,0.55),
        "3":(0.50,0.50),
        "4":(0.65,0.25),
        "5":(0.80,0.05),
        "6":(0.90,0.10)
    }
    return ifns[str(value)]

def m_operation(pair1, pair2):
    a1, b1 = pair1
    a2, b2 = pair2
    return (min(a1, a2), max(b1, b2))

def M_operation(pair1, pair2):
    a1, b1 = pair1
    a2, b2 = pair2
    return (max(a1, a2), min(b1, b2))


def dist(pair1, pair2):
    a1, b1 = pair1
    a2, b2 = pair2
    h1 = 1 - (a1 + b1)
    h2 = 1 - (a2 + b2)

    term1 = (abs(2*(a1 - a2) - (b1 - b2)) / 3) * (1 - (h1 + h2) / 2)
    term2 = (abs(2*(b1 - b2) - (a1 - a2)) / 3) * ((h1 + h2) / 2)

    return term1 + term2

### Input Experts Data

In [ ]:
df = pd.read_csv('input.csv')

# Calculate n
n = int(np.sqrt(df.shape[0]))
k = df.shape[1] - 1
# # Initialize the list of expert matrices
expert_matrices = []

# Iterate over the columns of the DataFrame
for i in range(1, df.shape[1]):
    # Convert the column values to a list
    column_values = df.iloc[:, i].tolist()

    # Reshape the list into an n*n matrix
    matrix = np.reshape(column_values, (n, n),order='F')
    matrix = matrix.T
    # Add the matrix to the list
    expert_matrices.append(matrix)

    # Convert matrices to IFN matrices
ifn_matrices = []

for matrix in expert_matrices:
    ifn_matrix = np.array([[generate_ifn(value) for value in row] for row in matrix])
    ifn_matrices.append(ifn_matrix)

# Write the matrices and IFN matrices to a file
with open("solution.txt", "w") as file:
    for i, (matrix, ifn_matrix) in enumerate(zip(expert_matrices, ifn_matrices), 1):
        file.write(f"Expert Matrix {i}:\n")
        for row in matrix:
            file.write(" ".join(map(str, row)) + "\n")
        file.write("\n")

        file.write(f"Expert {i} IFN Matrix :\n")
        for row in ifn_matrix:
            file.write(" ".join([f"({m:.2f}, {n:.2f})" for m, n in row]) + "\n")
        file.write("\n")

print("Matrices and IFN matrices have been written to 'solution.txt'")

Matrices and IFN matrices have been written to 'solution.txt'


In [ ]:
# priority_parameter=float(input("Enter the Priority Parameter: "))
priority_parameter=0.5

### Calculate IFPIO  Matrix

In [ ]:
IFPIO = np.zeros((n, n, 2))  # Initialize a nxn matrix with pairs
for i in range(n):
    for j in range(n):
        result = [1, 1]  # Initialize with neutral element for the $ operation
        for ifn_matrix in ifn_matrices:
            result = [result[0] * (ifn_matrix[i][j][0]**(1/k)),result[1]*((1-ifn_matrix[i][j][1])**(1/k))]
        IFPIO[i][j] = [result[0],1-result[1]]  # Take the average

# Append the IFPIO matrix to the solution.txt file
with open("solution.txt", "a") as file:
    file.write("IFPIO Matrix (Intuitionistic Fuzzy Positive Ideal Opinion Matrix):\n")
    for row in IFPIO:
        file.write(" ".join([f"({m:10.5f}, {n:10.5f})" for m, n in row]) + "\n")
    file.write("\n")

print("IFPIO matrix has been appended to 'solution.txt'")


IFPIO matrix has been appended to 'solution.txt'


### Calculate IFNIO Matrice

In [ ]:
# Compute the IFNIO matrix by swapping the pairs from the IFPIO matrix
IFNIO = np.array([[(b, a) for a, b in row] for row in IFPIO])

# Append the IFNIO matrix to the solution.txt file
with open("solution.txt", "a") as file:
    file.write("IFNIO Matrix (Intuitionistic Fuzzy Negative Ideal Opinion Matrix):\n")
    for row in IFNIO:
        file.write(" ".join([f"({m:10.5f}, {n:10.5f})" for m, n in row]) + "\n")
    file.write("\n")

print("IFNIO matrix has been appended to 'solution.txt'")


IFNIO matrix has been appended to 'solution.txt'


### Calculate IFLNIO Matrix

In [ ]:
# Compute the IFLNIO matrix using the m operation
IFLNIO = np.zeros((n, n, 2))  # Initialize a 5x5 matrix with pairs

for i in range(n):
    for j in range(n):
        result = ifn_matrices[0][i][j]  # Start with the first IFN matrix
        for p in range(1, len(ifn_matrices)):
            result = m_operation(result, ifn_matrices[p][i][j])
        IFLNIO[i][j] = result

# Append the IFLNIO matrix to the solution.txt file
with open("solution.txt", "a") as file:
    file.write("IFLNIO Matrix (Intuitionistic Fuzzy Left Negative Ideal Opinion Matrix):\n")
    for row in IFLNIO:
        file.write(" ".join([f"({m:10.5f}, {n:10.5f})" for m, n in row]) + "\n")
    file.write("\n")

print("IFLNIO matrix has been appended to 'solution.txt'")


IFLNIO matrix has been appended to 'solution.txt'


### Calculate IFRNIO Matrix

In [ ]:
# Compute the IFRNIO matrix using the M operation
IFRNIO = np.zeros((n, n, 2))  # Initialize a 5x5 matrix with pairs

for i in range(n):
    for j in range(n):
        result = ifn_matrices[0][i][j]  # Start with the first IFN matrix
        for p in range(1, len(ifn_matrices)):
            result = M_operation(result, ifn_matrices[p][i][j])
        IFRNIO[i][j] = result

# Append the IFRNIO matrix to the solution.txt file
with open("solution.txt", "a") as file:
    file.write("IFRNIO Matrix (Intuitionistic Fuzzy Right Negative Ideal Opinion Matrix):\n")
    for row in IFRNIO:
        file.write(" ".join([f"({m:10.5f}, {n:10.5f})" for m, n in row]) + "\n")
    file.write("\n")

print("IFRNIO matrix has been appended to 'solution.txt'")

IFRNIO matrix has been appended to 'solution.txt'


### Defined List of names of matrix

In [ ]:
matrices_list = [IFPIO, IFNIO, IFLNIO, IFRNIO]
matrices_names = ["IFPIO", "IFNIO", "IFLNIO", "IFRNIO"]

# List of the five IFN matrices
ifn_matrices_list = ifn_matrices
ifn_matrices_names = [f"IFN{p+1}" for p in range(k)]
expert_matrices_names = [f"Expert_{p+1}" for p in range(k)]
# Dictionary to store the distance matrices
distance_matrices_dict = {}

### Calculate distance matrix for all the possible combinations

In [ ]:
# Compute the distance matrices for all n*k combinations
with open("solution.txt", "a") as file:
    for i, matrix in enumerate(matrices_list):
          for j, ifn_matrix in enumerate(ifn_matrices_list):
              distance_matrix = np.zeros((n, n))
              for x in range(n):
                  for y in range(n):
                      distance_matrix[x][y] = dist(matrix[x][y], ifn_matrix[x][y])

              # Store the distance matrix in the dictionary
              key = f"{matrices_names[i]}_to_{ifn_matrices_names[j]}"

              distance_matrices_dict[key] = distance_matrix

              # Append the distance matrix to the solution.txt file
              file.write(f"Distance Matrix between {matrices_names[i]} and {ifn_matrices_names[j]}:\n")
              for row in distance_matrix:
                  file.write(" ".join([f"{value:10.5f}" for value in row]) + "\n")
              file.write("\n")

print("All distance matrices have been computed and stored.")

All distance matrices have been computed and stored.


### Initialize DistanceMatrix

In [ ]:
DistanceMatrix = np.zeros((k, 4))

### Calculate Values for DistanceMatrix

In [ ]:
# Compute the values for DistanceMatrix
for i, ifn_name in enumerate(ifn_matrices_names):
    for j, matrix_name in enumerate(matrices_names):
        key = f"{matrix_name}_to_{ifn_name}"
        DistanceMatrix[i][j] = np.sum(distance_matrices_dict[key])

# Append the DistanceMatrix to the solution.txt file
with open("solution.txt", "a") as file:
    file.write("DistanceMatrix (Summation of Distance Matrices):\n")

    # Top boundary
    file.write("+" + "-" * (len(matrices_names) * 12 + 7) + "+\n")

    # Column names with vertical boundaries
    file.write("|       " + " | ".join(matrices_names) + " |\n")

    # Separator line
    file.write("|" + "-" * (len(matrices_names) * 12 + 7) + "|\n")

    # Matrix rows with vertical boundaries
    for i, row in enumerate(DistanceMatrix):
        file.write("| " + expert_matrices_names[i] + " " + " | ".join([f"{value:10.5f}" for value in row]) + " |\n")

    # Bottom boundary
    file.write("+" + "-" * (len(matrices_names) * 12 + 7) + "+\n\n")

print("DistanceMatrix with a boundary has been appended to 'solution.txt'")

DistanceMatrix with a boundary has been appended to 'solution.txt'


### Calculate Closeness Coefficient

In [ ]:
closeness_coefficients = []

In [ ]:
for i in range(k):  # For each IFN
        numerator = sum(DistanceMatrix[i, j] for j in [1, 2, 3])  # Sum values from IFNIO, IFLNIO, and IFRNIO columns
        denominator = sum(DistanceMatrix[i])  # Sum values from all four columns
        coefficient = numerator / denominator if denominator != 0 else 0  # Handle potential division by zero
        closeness_coefficients.append(coefficient)

# Append the closeness coefficients to the solution.txt file
with open("solution.txt", "a") as file:
    file.write("Closeness Coefficients for each Expert:\n")
    for i, coefficient in enumerate(closeness_coefficients):
        file.write(f"{expert_matrices_names[i]}: {coefficient:10.5f}\n")
    file.write("\n")

print("Closeness coefficients have been computed and appended to 'solution.txt'")

Closeness coefficients have been computed and appended to 'solution.txt'


### Calculate **μ** [degree of membership] , **ν** [degree of non-membership] , **π** [degree of hesitation]



In [ ]:
mu_values = []
nu_values = []
hesitation_values = []

# Compute the values for each IFN
for i in range(k):
    # Compute mu value
    denominator = DistanceMatrix[i, 0] + DistanceMatrix[i, 1]
    mu = DistanceMatrix[i, 1] / denominator if denominator != 0 else 0
    mu_values.append(mu)

    # Compute nu value
    nu = 1 - closeness_coefficients[i]
    nu_values.append(nu)

    # Compute hesitation value
    hesitation = 1 - (mu + nu)
    hesitation_values.append(hesitation)

# Append the table to the solution.txt file
with open("solution.txt", "a") as file:
    file.write("Table of  μ, ν, and π values for each Expert:\n")
    file.write("IFN     |      μ     |   ν   | π\n")
    file.write("-" * 50 + "\n")
    for i in range(k):
        file.write(f"{expert_matrices_names[i]:<8}| {mu_values[i]:10.5f} | {nu_values[i]:10.5f} | {hesitation_values[i]:10.5f}\n")
    file.write("\n")

print("Table of  μ, ν, and π values has been appended to 'solution.txt'")

Table of  μ, ν, and π values has been appended to 'solution.txt'


### Weight For Each Expert

In [ ]:
# Compute the weights W for each IFN
weights = []
for i in range(k):
    mu = mu_values[i]
    nu = nu_values[i]
    hesitation = hesitation_values[i]
    weight = mu + hesitation * (mu / (mu + nu) if mu + nu != 0 else 0)  # Handle potential division by zero
    weights.append(weight)

# Compute the normalized weights NW
total_weight = sum(weights)
normalized_weights = [weight / total_weight if total_weight != 0 else 0 for weight in weights]  # Handle potential division by zero

# Append the weights and normalized weights to the solution.txt file
with open("solution.txt", "a") as file:
    file.write("Weights (W) and Normalized Weights (NW) for each Expert:\n")
    file.write("Expert     |     W      |     NW\n")
    file.write("-" * 40 + "\n")
    for i in range(k):
        file.write(f"{expert_matrices_names[i] :<8}| {weights[i]:10.5f} | {normalized_weights[i]:10.5f}\n")
    file.write("\n")

print("Weights and normalized weights have been computed and appended to 'solution.txt'")

Weights and normalized weights have been computed and appended to 'solution.txt'


### Inter-Relationship mining between factors

### Defining Function for getting weighted IFN matrix

In [ ]:
def multiply_matrix_and_weight(matrix, weight):
    """
    Multiplies each element of the matrix (each IFN) by the weight.
    Since the matrix contains tuples, we'll handle the multiplication element-wise for the tuples.
    """
    weighted_matrix = []
    for row in matrix:
        weighted_row = [(weight * element[0], weight * element[1]) for element in row]
        weighted_matrix.append(weighted_row)
    return weighted_matrix

### Get the weighted IFN matrices

In [ ]:
expert_weighted_ifn_matrices = [multiply_matrix_and_weight(matrix, weight) for matrix, weight in zip(ifn_matrices, normalized_weights)]

# Writing to the solution file
with open('solution.txt', 'a') as file:  # 'a' stands for append mode
    file.write("\nExpert Weighted IFN Matrices:\n")
    for i, matrix in enumerate(expert_weighted_ifn_matrices):
        file.write(f"\nMatric {i + 1}:\n")
        for row in matrix:
            # Writing each row in a readable format
            row_str = ', '.join([f"({x:.2f}, {y:.2f})" for x, y in row])
            file.write(f"{row_str}\n")

print("Expert weighted IFN matrices have been written to 'solution.txt'.")


Expert weighted IFN matrices have been written to 'solution.txt'.


### Generate Intuitionistic Fuzzy Weight Aggregated Inter-influence Matrix (IFWAIIM)

In [ ]:
def aggregate_with_dollar_operation(matrices,weights):
    """
    Aggregate multiple IFN matrices using the '$' operation.
    """
    aggregated_matrix = np.zeros((n,n,2))
    for i in range(n):
      for j in range(n):
        e = 0;
        result = [1, 1]  # Initialize with neutral element for the $ operation
        for ifn_matrix in ifn_matrices:
            result = [result[0] * (ifn_matrix[i][j][0]**(weights[e])),result[1]*((1-ifn_matrix[i][j][1])**weights[e])]
            e = e+1
        aggregated_matrix[i][j] = [result[0],1-result[1]]  # Take the average
    return aggregated_matrix


# Calculate the IFWAIIM
ifwaiim = aggregate_with_dollar_operation(ifn_matrices,normalized_weights)

# Writing to the solution file
with open('solution.txt', 'a') as file:  # 'a' stands for append mode
    file.write("\nIntuitionistic Fuzzy Weight Aggregated Inter-influence Matrix (IFWAIIM):\n")
    for row in ifwaiim:
        # Writing each row in a readable format
        row_str = ', '.join([f"({x:.2f}, {y:.2f})" for x, y in row])
        file.write(f"{row_str}\n")

print("IFWAIIM has been written to 'solution.txt'.")


IFWAIIM has been written to 'solution.txt'.


### Generate C-IIM (Crisp Inter-Infuence Matrix)

In [ ]:
def transform_to_crisp_value(ifn):
    """
    Transform an IFN to a crisp value using the formula (mu - nu + 1) / 2.
    """
    mu, nu = ifn
    return (mu - nu + 1) / 2

def create_ciim(ifwaiim):
    """
    Create the Crisp Inter-Influence Matrix (CIIM) by transforming the IFWAIIM.
    """
    ciim = []
    for row in ifwaiim:
        crisp_row = [transform_to_crisp_value(ifn) for ifn in row]
        ciim.append(crisp_row)
    return ciim


# Create the CIIM from the IFWAIIM
ciim = create_ciim(ifwaiim)

# Writing to the solution file
with open('solution.txt', 'a') as file:  # 'a' stands for append mode
    file.write("\nCrisp Inter-Influence Matrix (CIIM):\n")
    for row in ciim:
        # Writing each row in a readable format
        row_str = ', '.join([f"{value:.2f}" for value in row])
        file.write(f"{row_str}\n")

print("CIIM has been written to 'solution.txt'.")

CIIM has been written to 'solution.txt'.


### Determine the MAXSUM(m*)

In [ ]:
# Step 1: Find the maximum in each row and sum these maximums
m1 = np.max(np.sum(ciim, axis=1))

# Step 2: Find the maximum in each column and sum these maximums
m2 = np.max(np.sum(ciim, axis=0))

# Step 5: Determine the maximum between 'max_row_sum' and 'max_col_sum'
max_value_in_ciim = max(m1, m2)

    # Writing to the solution file
with open('solution.txt', 'a') as file:  # 'a' stands for append mode
    file.write("\n========================================\n")  # For emphasis and separation
    file.write(f"Maximum Value in CIIM: {max_value_in_ciim:.2f}\n")
    file.write("========================================\n")  # For emphasis and separation

print("Maximum value in CIIM has been written to 'solution.txt'.")

Maximum value in CIIM has been written to 'solution.txt'.


### Generate N-IIM (Normalized Inter-Infuence Matrix)

In [ ]:
n_iim = [[value / max_value_in_ciim for value in row] for row in ciim]

# Writing to the solution file
with open('solution.txt', 'a') as file:  # 'a' stands for append mode
    file.write("\nNormalized Inter-Influence Matrix (N-IIM):\n")
    for row in n_iim:
        # Writing each row in a readable format
        row_str = ', '.join([f"{value:.2f}" for value in row])
        file.write(f"{row_str}\n")

print("N-IIM has been written to 'solution.txt'.")

N-IIM has been written to 'solution.txt'.


### Generate T-IIM (Total Inter-Infuence Matrix)

In [ ]:
# Generate an identity matrix of size n x n
identity_matrix = np.identity(n)

# Calculate (I - n_iim)
subtract_matrix = identity_matrix - n_iim
try:
    # Calculate the inverse of (I - n_iim)
    inverse_matrix = np.linalg.inv(subtract_matrix)
except np.linalg.LinAlgError:
    # Handle the case where the matrix is not invertible
    print("Error: (I - n_iim) matrix is not invertible.")
    exit()

with open('solution.txt', 'a') as file:  # 'a' stands for append mode
        file.write("\nInverse of (I - n_iim):\n")
        for row in inverse_matrix:
            row_str = ', '.join([f"{value:.2f}" for value in row])
            file.write(f"[{row_str}]\n")

# Calculate T-IIM: n_iim * (I - n_iim)^-1
t_iim =n_iim @ inverse_matrix

# Writing to the solution file
with open('solution.txt', 'a') as file:  # 'a' stands for append mode
    file.write("\nTotal Inter-Influence Matrix (T-IIM):\n")
    for row in t_iim:
        # Writing each row in a readable format
        row_str = ', '.join([f"{value:.2f}" for value in row])
        file.write(f"{row_str}\n")

print("T-IIM has been written to 'solution.txt'.")




T-IIM has been written to 'solution.txt'.


### Calculating  **α** , **minimum and maximum**



In [ ]:
# Calculate the average of all elements in the matrix.
α = np.mean(t_iim)+(np.std(t_iim)/2)
# Find the maximum and minimum values in the matrix
max_t_iim = np.max(t_iim)
min_t_iim = np.min(t_iim)


with open('solution.txt', 'a') as file:  # 'a' stands for append mode
    file.write("\n========================================\n")  # For emphasis and separation
    file.write(f"Threshold : {α:.4f}\n")
    file.write("========================================\n")  # For emphasis and separation
    print("Threshold α has been written to 'solution.txt'.")
    file.write("\n========================================\n")  # For emphasis and separation
    file.write(f"Minimum : {min_t_iim:.4f}\n")
    file.write("========================================\n")  # For emphasis and separation
    print("Minimum value has been written to 'solution.txt'.")
    file.write("\n========================================\n")  # For emphasis and separation
    file.write(f"Maximum : {max_t_iim:.4f}\n")
    file.write("========================================\n")  # For emphasis and separation
    print("Maximum value has been written to 'solution.txt'.")




Threshold α has been written to 'solution.txt'.
Minimum value has been written to 'solution.txt'.
Maximum value has been written to 'solution.txt'.


### Modified Total Inter-Influence Matrix

In [ ]:
# prompt: generate a identity matrix of size similar to t_iim and then sum it with t_iim

identity_matrix = np.identity(n)
modified_t_iim = t_iim + identity_matrix
# Writing to the solution file
with open('solution.txt', 'a') as file:  # 'a' stands for append mode
    file.write("\nModified Total Inter-Influence Matrix:\n")
    for row in modified_t_iim:
        # Writing each row in a readable format
        row_str = ', '.join([f"{value:.2f}" for value in row])
        file.write(f"{row_str}\n")

print("Modified T-IIM has been written to 'solution.txt'.")


Modified T-IIM has been written to 'solution.txt'.


### Calculating Reachability Matrix

In [ ]:
# prompt: if modified_t_iim value is greater than alpha then 1 else 0 ... call it reachable matrix

reachable_matrix = np.where(modified_t_iim >= α, 1, 0)

# Writing to the solution file
with open('solution.txt', 'a') as file:  # 'a' stands for append mode
    file.write("\nReachability Matrix:\n")
    for row in reachable_matrix:
        # Writing each row in a readable format
        row_str = ', '.join([f"{value}" for value in row])
        file.write(f"{row_str}\n")

print("Reachability Matrix has been written to 'solution.txt'.")


Reachability Matrix has been written to 'solution.txt'.


### Calculate #influenced, #influent_DA, DAI


In [ ]:
influenced= np.zeros(n, dtype=int)
influent_DA= np.zeros(n, dtype=int)
# Generate a random Directly Accessible Index (DAI) list with binary values
DAI = [1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,0]
# Count the number of directly accessible factors influencing each factor
for i in range(n):
        for j in range(n):
            if reachable_matrix[i, j] == 1 and i!=j:
                    influenced[i] += 1
            if reachable_matrix[j,i] == 1 and i!=j and DAI[j]==1:
                    influent_DA[i] += 1

# Write the results to the file in a tabular form
with open('solution.txt', 'a') as file:
     file.write("+--------+-----+-------------+-------------+\n")
     file.write("| Factor | DAI | #Influenced | #Influent_DA |\n")
     file.write("+--------+-----+-------------+-------------+\n")
     for i in range(n):
        file.write(f"| {i+1:<6} | {DAI[i]:<3} | {influenced[i]:<11} | {influent_DA[i]:<11} |\n")
        file.write("+--------+-----+-------------+-------------+\n")

### Initial level Partitioning

In [ ]:
df = pd.DataFrame({
        'Factor': np.arange(1,n+1),
        'DAI': DAI,
        '#Influenced': influenced,
        '#Influent_DA': influent_DA
    })

# Sorting based on the rules provided
df.sort_values(by=['#Influent_DA', '#Influenced', 'DAI'], ascending=[True, True, False], inplace=True)

 # Assign levels and sub-levels
df['Level'] = None
current_level = 1
sublevel_char = 'a'
df = df.reset_index(drop=True);
for i in range(n):
    if i > 0:
        same_influent_DA = df.iloc[i]['#Influent_DA'] == df.iloc[i - 1]['#Influent_DA']
        same_influenced = df.iloc[i]['#Influenced'] == df.iloc[i - 1]['#Influenced']
        same_DAI = df.iloc[i]['DAI'] == df.iloc[i - 1]['DAI']
        # print(same_influent_DA,same_influenced,same_DAI)
        if not same_influent_DA:
            current_level += 1
            sublevel_char = 'a'
        elif same_influenced and same_DAI:
            # same sub-level if all are equal
            pass
        else:
            # next sub-level (a, b, c, ...)
            sublevel_char = chr(ord(sublevel_char) + 1)
    df.at[i,"Level"] = f"{current_level}-{sublevel_char}"
    # print(df.at[i, 'Level'])
df.sort_values(by=['Factor'], ascending=[True], inplace=True)
df = df.reset_index(drop=True);
print(df)
with open("solution.txt", 'a') as file:
        file.write("Initial Level Partitioning\n")
        file.write(df.to_string(index=False))
        file.write("\n\n")
print("Solution with levels has been written to 'solution_with_levels.txt'.")

    Factor  DAI  #Influenced  #Influent_DA Level
0        1    1            3             3   4-b
1        2    1           12            10   9-a
2        3    1            7             4   5-b
3        4    1            0             0   1-a
4        5    1            0             0   1-a
5        6    0            7             8   7-a
6        7    1            4             2   3-a
7        8    1            3             0   1-b
8        9    1            9             9   8-a
9       10    1            5             4   5-a
10      11    1           11             5   6-b
11      12    1            2             3   4-a
12      13    1            8             5   6-a
13      14    1            0             0   1-a
14      15    1           10             2   3-b
15      16    1            5             1   2-a
16      17    0           14            11  10-a
17      18    0           14            12  11-a
Solution with levels has been written to 'solution_with_levels.txt'.


### Iterative Level Partitioning



In [ ]:
# returns 1 if val1>val2 , -1 if val1<val2 and 0 if both are equal
def compare_values(val1, val2):
    # Splitting the values into numeric and alphabetic parts
    num1, alpha1 = val1.split('-')
    num2, alpha2 = val2.split('-')

    # Convert numeric parts to integers for comparison
    num1, num2 = int(num1), int(num2)

    # Compare numeric parts
    if num1 > num2:
        return 1
    elif num1 < num2:
        return -1
    else:
        # If numeric parts are equal, compare alphabetic parts
        if alpha1 > alpha2:
            return 1
        elif alpha1 < alpha2:
            return -1
        else:
            return 0


def recalculate_values(df, RM):
    # Recalculating #influent_DA and #influenced based on the current level partitioning
    n = len(df)
    new_influent_DA = np.zeros(n, dtype=int)
    new_influenced = np.zeros(n, dtype=int)

    for i in range(n):
        for j in range(n):
            if RM[i, j] == 1 and i!=j:
                if compare_values(df.at[i, 'Level'] , df.at[j, 'Level'])==-1:
                    new_influenced[i] += 1
            if RM[j,i] == 1 and i!=j:
                if compare_values(df.at[i, 'Level'] ,df.at[j, 'Level'])==1and DAI[j]==1:
                    new_influent_DA[i] += 1

    df['#Influent_DA'] = new_influent_DA
    df['#Influenced'] = new_influenced
    return df

def assign_levels(df):
  # Sorting and assigning levels
  df.sort_values(by=['#Influent_DA', '#Influenced', 'DAI'], ascending=[True, True, False], inplace=True)
  df['Level'] = None
  current_level = 1
  sublevel_char = 'a'
  df = df.reset_index(drop=True)
  for i in range(n):
    if i > 0:
        same_influent_DA = df.iloc[i]['#Influent_DA'] == df.iloc[i - 1]['#Influent_DA']
        same_influenced = df.iloc[i]['#Influenced'] == df.iloc[i - 1]['#Influenced']
        same_DAI = df.iloc[i]['DAI'] == df.iloc[i - 1]['DAI']
        # print(same_influent_DA,same_influenced,same_DAI)
        if not same_influent_DA:
            current_level += 1
            sublevel_char = 'a'
        elif same_influenced and same_DAI:
            # same sub-level if all are equal
            pass
        else:
            # next sub-level (a, b, c, ...)
            sublevel_char = chr(ord(sublevel_char) + 1)
    df.at[i,"Level"] = f"{current_level}-{sublevel_char}"
    # print(df.at[i, 'Level'])
  df.sort_values(by=['Factor'], ascending=[True], inplace=True)
  df = df.reset_index(drop=True);
  return df

def perform_iterations(df, RM):
  iteration = 0
  history = []
  i = 1
  while i<20:
      df = recalculate_values(df, RM)
      df = assign_levels(df)
      iteration += 1

      # Check if the level partitioning has stabilized
      if iteration>1:
        print(history[-1])

      if iteration > 1 and df.equals(history[-1]):
          break

      history.append(df.copy())
      i = i+1;

  return history

iterations_history = perform_iterations(df, reachable_matrix)
df = iterations_history[-1]
# Writing the results to 'solution.txt'
with open('solution.txt', 'a') as file:
    for i, iteration_df in enumerate(iterations_history, start=1):
        file.write(f"Iteration {i}\n")
        file.write(iteration_df.to_string(index=False))
        file.write("\n\n")



    Factor  DAI  #Influenced  #Influent_DA Level
0        1    1            3             1   2-a
1        2    1            2            10   7-a
2        3    1            7             1   2-c
3        4    1            0             0   1-a
4        5    1            0             0   1-a
5        6    0            4             6   5-a
6        7    1            4             0   1-d
7        8    1            3             0   1-c
8        9    1            3             8   6-a
9       10    1            5             1   2-b
10      11    1            5             3   4-a
11      12    1            2             0   1-b
12      13    1            6             2   3-a
13      14    1            0             0   1-a
14      15    1           10             0   1-f
15      16    1            5             0   1-e
16      17    0            1            11   8-a
17      18    0            0            12   9-a


# Factor Ranking Stage

## Sub level depth of factors

In [ ]:
def custom_sort(level):
    parts = level.split('-')
    number = int(parts[0])
    letter = parts[1] if len(parts) > 1 else ''
    return (number, letter)

sorted_df = df.sort_values(by='Level', key=lambda x: x.map(custom_sort))
def assign_consecutive_levels(df, level_column='Level'):
    """
    Assigns consecutive integer levels to a DataFrame based on a specified level column.
    This function assumes that the DataFrame is already sorted by the level column.
    """
    unique_levels = df[level_column].unique()
    level_mapping = {level: i+1 for i, level in enumerate(unique_levels)}

    df['SubLevelDepth'] = df[level_column].map(level_mapping)
    print(df)
    df.sort_values(by=['Factor'], ascending=[True], inplace=True)
    return df
df = assign_consecutive_levels(sorted_df)
with open('solution.txt', 'a') as file:
      file.write(f"Sub Level Depth Of Factors\n")
      file.write(df.to_string(index=False))
      file.write("\n\n")
print(df)

    Factor  DAI  #Influenced  #Influent_DA Level  SubLevelDepth
3        4    1            0             0   1-a              1
4        5    1            0             0   1-a              1
13      14    1            0             0   1-a              1
11      12    1            2             0   1-b              2
7        8    1            3             0   1-c              3
6        7    1            4             0   1-d              4
15      16    1            5             0   1-e              5
14      15    1           10             0   1-f              6
0        1    1            3             1   2-a              7
9       10    1            5             1   2-b              8
2        3    1            7             1   2-c              9
12      13    1            6             2   3-a             10
10      11    1            5             3   4-a             11
5        6    0            4             6   5-a             12
8        9    1            3            

## Upper Level Drive (uld) and Lower Level Dependence (lldp)

In [ ]:
def influeneced_higher_depth_TIIM(df, TIIM):
    sums = []
    for index, row in df.iterrows():
        current_level = row['Level']
        # Sum TIIM values for factors influenced by the current factor and having a higher level
        sum_val = sum(TIIM[row['Factor'] - 1, i] for i in range(len(df)) if df.at[i, 'Level'] > current_level)
        sums.append(sum_val)
    return sums

def influent_lower_depth_TIIM(df, TIIM):
    sums = []
    for index, row in df.iterrows():
        current_level = row['Level']
        # Sum TIIM values for factors influenced by the current factor and having a higher level
        sum_val = sum(TIIM[i,row['Factor'] - 1] for i in range(len(df)) if df.at[i, 'Level'] < current_level)
        sums.append(sum_val)
    return sums
# Apply the function to the DataFrame
df['ULD'] = influeneced_higher_depth_TIIM(df, t_iim)
df['LLD'] = influent_lower_depth_TIIM(df, t_iim)
print(df)

    Factor  DAI  #Influenced  #Influent_DA Level  SubLevelDepth       ULD  \
0        1    1            3             1   2-a              7  1.623923   
1        2    1            2            10   7-a             14  0.476587   
2        3    1            7             1   2-c              9  1.438237   
3        4    1            0             0   1-a              1  1.616010   
4        5    1            0             0   1-a              1  1.991570   
5        6    0            4             6   5-a             12  0.836452   
6        7    1            4             0   1-d              4  2.079077   
7        8    1            3             0   1-c              3  2.183703   
8        9    1            3             8   6-a             13  0.654545   
9       10    1            5             1   2-b              8  1.525301   
10      11    1            5             3   4-a             11  1.136714   
11      12    1            2             0   1-b              2  2.214701   

### Total Drive and Total Dependence of Factors

In [ ]:
def total_drive(df, TIIM):
    sums = []
    for index, row in df.iterrows():
        # Sum TIIM values for factors influenced by the current factor and having a higher level
        sum_val = sum(TIIM[row['Factor'] - 1, i] for i in range(len(df)))
        sums.append(sum_val)
    return sums

def total_dependence(df, TIIM):
    sums = []
    for index, row in df.iterrows():
        current_level = row['Level']
        # Sum TIIM values for factors influenced by the current factor and having a higher level
        sum_val = sum(TIIM[i,row['Factor'] - 1] for i in range(len(df)))
        sums.append(sum_val)
    return sums

df['TotalDrive'] = total_drive(df,t_iim)
df['TotalDependence'] = total_dependence(df,t_iim)

### Sub Level Index (SLI) Calculation

In [ ]:

df['SLI'] = [DF/(n-DF+1) for DF in df['SubLevelDepth']]

sli_counts = df['SLI'].value_counts()
multi_occurrence_sli = sli_counts[sli_counts > 1].index

def correct_sli(x):
  DF = df['SubLevelDepth'][x]
  k = df['SLI'][x] + 0.5*(((DF+1)/(n-DF))-df['SLI'][x])*((df['ULD'][x]/df['TotalDrive'][x])+(df['LLD'][x]/df['TotalDependence'][x]) )
  return k
# Apply the formula to elements with more than one occurrence, keep original values for others
df['IMP_F'] = [correct_sli(i) if df.at[i, 'SLI'] in multi_occurrence_sli else df.at[i, 'SLI'] for i in df.index]
df_sorted_for_ranking = df.sort_values(by='IMP_F', ascending=False)
df_sorted_for_ranking['Rank'] = range(1, len(df) + 1)

# Add the rank back to the original DataFrame
df = df.join(df_sorted_for_ranking['Rank'], how='left')
with open('solution.txt', 'a') as file:
      file.write(f"Importance Of Factors\n")
      file.write(df.to_string(index=False))
      file.write("\n\n")
print(df)

    Factor  DAI  #Influenced  #Influent_DA Level  SubLevelDepth       ULD  \
0        1    1            3             1   2-a              7  1.623923   
1        2    1            2            10   7-a             14  0.476587   
2        3    1            7             1   2-c              9  1.438237   
3        4    1            0             0   1-a              1  1.616010   
4        5    1            0             0   1-a              1  1.991570   
5        6    0            4             6   5-a             12  0.836452   
6        7    1            4             0   1-d              4  2.079077   
7        8    1            3             0   1-c              3  2.183703   
8        9    1            3             8   6-a             13  0.654545   
9       10    1            5             1   2-b              8  1.525301   
10      11    1            5             3   4-a             11  1.136714   
11      12    1            2             0   1-b              2  2.214701   